# Phe vs Roche Model Comparison

In this notebook we use the `epimodels` module to investigate how non-pharmaceutical interventions impact the outcome of the epidemic. We use two models: the _PHE model_ by niversity of Cambridge and the _Roche model_ by F. Hoffmann-La Roche Ltd, and run a forward simulation simulation both with and without NPIs, after they have been fitted to serology data.

The analyses are run for:
 - Dates: **15 Feb 2020** - **15 May 2020**;
 - PHE regions of interest: **London**.

We use realistic serology and mortality data extracted from the REACT survey and GOV.UK data.

*The PHE model is built by Public Health England in collaboration with University of Cambridge. The Roche model is built by F. Hoffmann-La Roche Ltd.*

In [1]:
# Load necessary libraries
import os
import numpy as np
import pandas as pd
from scipy.stats import gamma, nbinom
import epimodels as em
import matplotlib
import plotly.graph_objects as go
import plotly.express as px
from matplotlib import pyplot as plt
from iteration_utilities import deepflatten

## Model Setup
### Define setup matrices for the PHE and Roche Model + NPIs

In [2]:
# Populate the model
total_days =  90
regions = ['London']
age_groups = ['0-1', '1-5', '5-15', '15-25', '25-45', '45-65', '65-75', '75+']

weeks = list(range(1,int(np.ceil(total_days/7))+1))

### Variable
matrices_region_var = []

# Initial state of the system
for w in weeks:
    weeks_matrices_region_var = []
    for r in regions:
        path = os.path.join('../../data/final_contact_matrices/{}_W{}.csv'.format(r, w))
        region_data_matrix_var = pd.read_csv(path, header=None, dtype=np.float64)
        regional_var = em.RegionMatrix(r, age_groups, region_data_matrix_var)
        weeks_matrices_region_var.append(regional_var)

    matrices_region_var.append(weeks_matrices_region_var)

contacts_var = em.ContactMatrix(age_groups, np.ones((len(age_groups), len(age_groups))))
matrices_contact_var = [contacts_var]

# Matrices contact
time_changes_contact_var = [1]
time_changes_region_var = np.arange(1, total_days+1, 7).tolist()

### Fixed
matrices_region_fix = []

# Initial state of the system
weeks_matrices_region_fix = []
for r in regions:
    path = os.path.join('../../data/final_contact_matrices/BASE.csv')
    region_data_matrix_fix = pd.read_csv(path, header=None, dtype=np.float64)
    regional_fix = em.RegionMatrix(r, age_groups, region_data_matrix_fix)
    weeks_matrices_region_fix.append(regional_fix)

matrices_region_fix.append(weeks_matrices_region_fix)

contacts_fix = em.ContactMatrix(age_groups, np.ones((len(age_groups), len(age_groups))))
matrices_contact_fix = [contacts_fix]

# Matrices contact
time_changes_contact_fix = [1]
time_changes_region_fix = [1]

### With NPIs data
max_levels_npi = [3, 3, 2, 4, 2, 3, 2, 4, 2]
targeted_npi = [True, True, True, True, True, True, True, False, True]
path = os.path.join('../../data/npi_data/')
general_npi_yes = np.loadtxt(os.path.join(path, 'uk_flags.csv'), dtype=bool, delimiter=',').tolist()
time_changes_flag_yes = np.loadtxt(os.path.join(path, 'times_flags.csv'), dtype=int, delimiter=',').tolist()

reg_levels_npi_yes = [np.loadtxt(os.path.join(path, 'uk_npis.csv'), dtype=int, delimiter=',').tolist()]
time_changes_npi_yes = np.loadtxt(os.path.join(path, 'times_npis.csv'), dtype=int, delimiter=',').tolist()

### No NPIs
general_npi_no = [[False, False, False, False, False, False, False, False, False]]
time_changes_flag_no = [1]

reg_levels_npi_no = [[0, 0, 0, 0, 0, 0, 0, 0, 0]]
time_changes_npi_no = [1]


### Set the parameters and initial conditions of the model and bundle everything together

### PHE with variable contacts

In [ ]:
# Instantiate model
phe_model_var = em.PheSEIRModel()

# Set the region names, age groups, contact and regional data of the model
phe_model_var.set_regions(regions)
phe_model_var.set_age_groups(age_groups)
phe_model_var.read_contact_data(matrices_contact_var, time_changes_contact_var)
phe_model_var.read_regional_data(matrices_region_var, time_changes_region_var)

# Initial number of susceptibles
path = os.path.join('../../data/england_population/England_population.csv')
total_susceptibles = np.loadtxt(path, dtype=int, delimiter=',').tolist()
susceptibles = total_susceptibles[1]

# Initial number of infectives
ICs_multiplier = 50
infectives1 = (ICs_multiplier * np.ones((len(regions), len(age_groups)))).tolist()

infectives2 = np.zeros((len(regions), len(age_groups))).tolist()

dI = 4
dL = 4

# List of times at which we wish to evaluate the states of the compartments of the phe_model_var
times = np.arange(1, total_days+1, 1).tolist()

In [ ]:
### Variable Model
# Set regional and time dependent parameters
phe_model_var_regional_parameters = em.PheRegParameters(
    model=phe_model_var,
    initial_r=initial_r,
    region_index=4,
    betas=np.ones((len(regions), len(times))).tolist(),
    times=times
)

# Set ICs parameters
phe_model_var_ICs = em.PheICs(
    model=phe_model_var,
    susceptibles_IC=susceptibles,
    exposed1_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    exposed2_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    infectives1_IC=infectives1,
    infectives2_IC=infectives2,
    recovered_IC=np.zeros((len(regions), len(age_groups))).tolist()
)

# Set disease-specific parameters
phe_model_var_disease_parameters = em.PheDiseaseParameters(
    model=phe_model_var,
    dL=dL,
    dI=dI
)

# Set other simulation parameters
phe_model_var_simulation_parameters = em.PheSimParameters(
    model=phe_model_var,
    delta_t=0.5,
    method='RK45'
)

# Set all parameters in the controller
phe_model_var_parameters = em.PheParametersController(
    model=phe_model_var,
    regional_parameters=phe_model_var_regional_parameters,
    ICs=phe_model_var_ICs,
    disease_parameters=phe_model_var_disease_parameters,
    simulation_parameters=phe_model_var_simulation_parameters
)

### PHE with fixed contacts

In [ ]:
# Instantiate phe_model_var
phe_model_fix = em.PheSEIRModel()

# Set the region names, age groups, contact and regional data of the model
phe_model_fix.set_regions(regions)
phe_model_fix.set_age_groups(age_groups)
phe_model_fix.read_contact_data(matrices_contact_fix, time_changes_contact_fix)
phe_model_fix.read_regional_data(matrices_region_fix, time_changes_region_fix)

# Initial number of susceptibles
path = os.path.join('../../data/england_population/England_population.csv')
total_susceptibles = np.loadtxt(path, dtype=int, delimiter=',').tolist()
susceptibles = total_susceptibles[1]

# Initial number of infectives
ICs_multiplier = 50
infectives1 = (ICs_multiplier * np.ones((len(regions), len(age_groups)))).tolist()

infectives2 = np.zeros((len(regions), len(age_groups))).tolist()

dI = 4
dL = 4

# List of times at which we wish to evaluate the states of the compartments of the model
times = np.arange(1, total_days+1, 1).tolist()

In [ ]:
### Fixed Model
# Set regional and time dependent parameters
phe_model_fix_regional_parameters = em.PheRegParameters(
    model=phe_model_fix,
    initial_r=initial_r,
    region_index=4,
    betas=np.ones((len(regions), len(times))).tolist(),
    times=times
)

# Set ICs parameters
phe_model_fix_ICs = em.PheICs(
    model=phe_model_fix,
    susceptibles_IC=susceptibles,
    exposed1_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    exposed2_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    infectives1_IC=infectives1,
    infectives2_IC=infectives2,
    recovered_IC=np.zeros((len(regions), len(age_groups))).tolist()
)

# Set disease-specific parameters
phe_model_fix_disease_parameters = em.PheDiseaseParameters(
    model=phe_model_fix,
    dL=dL,
    dI=dI
)

# Set other simulation parameters
phe_model_fix_simulation_parameters = em.PheSimParameters(
    model=phe_model_fix,
    delta_t=0.5,
    method='RK45'
)

# Set all parameters in the controller
phe_model_fix_parameters = em.PheParametersController(
    model=phe_model_fix,
    regional_parameters=phe_model_fix_regional_parameters,
    ICs=phe_model_fix_ICs,
    disease_parameters=phe_model_fix_disease_parameters,
    simulation_parameters=phe_model_fix_simulation_parameters
)

### Roche with NPIs

In [ ]:
# Instantiate model
roche_model_var = em.RocheSEIRModel()

# Set the region names, contact and regional data of the model
roche_model_var.set_regions(regions)
roche_model_var.set_age_groups(age_groups)
roche_model_var.read_contact_data(matrices_contact_fix, time_changes_contact_fix)
roche_model_var.read_regional_data(matrices_region_fix, time_changes_region_fix)
roche_model_var.read_npis_data(max_levels_npi, targeted_npi, general_npi_yes, reg_levels_npi_yes, time_changes_npi_yes, time_changes_flag_yes)

# Initial number of susceptibles
path = os.path.join('../../data/england_population/England_population.csv')
total_susceptibles = np.loadtxt(path, dtype=int, delimiter=',').tolist()
susceptibles = []
susceptibles.append(total_susceptibles[-1])

# Initial number of infectives
ICs_multiplier = [30, 5]
infectives_pre = (ICs_multiplier[0] * np.ones((len(regions), len(age_groups)))).tolist()
infectives_pre_ss = (ICs_multiplier[1] * np.ones((len(regions), len(age_groups)))).tolist()

infectives = (np.array(infectives_pre) + np.array(infectives_pre_ss)).tolist()

# List of times at which we wish to evaluate the states of the compartments of the model
times = np.arange(1, total_days+1, 1).tolist()

In [ ]:
# Set ICs parameters
roche_model_var_ICs = em.RocheICs(
    model=roche_model_var,
    susceptibles_IC=susceptibles,
    exposed_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    infectives_pre_IC=infectives_pre,
    infectives_asym_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    infectives_sym_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    infectives_pre_ss_IC=infectives_pre_ss,
    infectives_asym_ss_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    infectives_sym_ss_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    infectives_q_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    recovered_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    recovered_asym_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    dead_IC=np.zeros((len(regions), len(age_groups))).tolist()
)

# Set average times in compartments
roche_model_var_compartment_times = em.RocheCompartmentTimes(
    model=roche_model_var,
    k=2.59,
    kS=4.28,
    kQ=1,
    kR=9,
    kRI=10
)

# Set proportion of asymptomatic, super-spreader and dead cases
roche_model_var_proportion_parameters = em.RocheProportions(
    model=roche_model_var,
    Pa = 0.716,
    Pss = 0.106,
    Pd = 0.05
)

# Set transmission parameters
roche_model_var_transmission_parameters = em.RocheTransmission(
    model=roche_model_var,
    beta_min=0.228,
    beta_max=2.63,
    bss=3.11,
    gamma=1,
    s50=34.9
)

# Set other simulation parameters
roche_model_var_simulation_parameters = em.RocheSimParameters(
    model=roche_model_var,
    region_index=1,
    method='RK45',
    times=times
)

# Set all parameters in the controller
roche_model_var_parameters = em.RocheParametersController(
    model=roche_model_var,
    ICs=roche_model_var_ICs,
    compartment_times=roche_model_var_compartment_times,
    proportion_parameters=roche_model_var_proportion_parameters,
    transmission_parameters=roche_model_var_transmission_parameters,
    simulation_parameters=roche_model_var_simulation_parameters
)

### Roche without NPIs

In [ ]:
# Instantiate model
roche_model_fix = em.RocheSEIRModel()

# Set the region names, contact and regional data of the model
roche_model_fix.set_regions(regions)
roche_model_fix.set_age_groups(age_groups)
roche_model_fix.read_contact_data(matrices_contact_fix, time_changes_contact_fix)
roche_model_fix.read_regional_data(matrices_region_fix, time_changes_region_fix)
roche_model_fix.read_npis_data(max_levels_npi, targeted_npi, general_npi_no, reg_levels_npi_no, time_changes_npi_no, time_changes_flag_no)

# Initial number of susceptibles
path = os.path.join('../../data/england_population/England_population.csv')
total_susceptibles = np.loadtxt(path, dtype=int, delimiter=',').tolist()
susceptibles = []
susceptibles.append(total_susceptibles[-1])

# Initial number of infectives
ICs_multiplier = [30, 5]
infectives_pre = (ICs_multiplier[0] * np.ones((len(regions), len(age_groups)))).tolist()
infectives_pre_ss = (ICs_multiplier[1] * np.ones((len(regions), len(age_groups)))).tolist()

infectives = (np.array(infectives_pre) + np.array(infectives_pre_ss)).tolist()

# List of times at which we wish to evaluate the states of the compartments of the model
times = np.arange(1, total_days+1, 1).tolist()

In [ ]:
# Set ICs parameters
roche_model_fix_ICs = em.RocheICs(
    model=roche_model_fix,
    susceptibles_IC=susceptibles,
    exposed_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    infectives_pre_IC=infectives_pre,
    infectives_asym_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    infectives_sym_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    infectives_pre_ss_IC=infectives_pre_ss,
    infectives_asym_ss_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    infectives_sym_ss_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    infectives_q_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    recovered_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    recovered_asym_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    dead_IC=np.zeros((len(regions), len(age_groups))).tolist()
)

# Set average times in compartments
roche_model_fix_compartment_times = em.RocheCompartmentTimes(
    model=roche_model_fix,
    k=2.59,
    kS=4.28,
    kQ=1,
    kR=9,
    kRI=10
)

# Set proportion of asymptomatic, super-spreader and dead cases
roche_model_fix_proportion_parameters = em.RocheProportions(
    model=roche_model_fix,
    Pa = 0.716,
    Pss = 0.106,
    Pd = 0.05
)

# Set transmission parameters
roche_model_fix_transmission_parameters = em.RocheTransmission(
    model=roche_model_fix,
    beta_min=0.228,
    beta_max=2.63,
    bss=3.11,
    gamma=1,
    s50=34.9
)

# Set other simulation parameters
roche_model_fix_simulation_parameters = em.RocheSimParameters(
    model=roche_model_fix,
    region_index=1,
    method='RK45',
    times=times
)

# Set all parameters in the controller
roche_model_fix_parameters = em.RocheParametersController(
    model=roche_model_fix,
    ICs=roche_model_fix_ICs,
    compartment_times=roche_model_fix_compartment_times,
    proportion_parameters=roche_model_fix_proportion_parameters,
    transmission_parameters=roche_model_fix_transmission_parameters,
    simulation_parameters=roche_model_fix_simulation_parameters
)

## Death and Serology data
### Read Death and Serology data

In [ ]:
# Read in death and positive data from external files
deaths_data = []
positives_data = []
tests = []

for region in regions:
    deaths_data.append(np.loadtxt('../../data/death_data/{}_deaths.csv'.format(region), dtype=int, delimiter=','))
    positives_data.append(np.loadtxt('../../data/serology_data/{}_positives_nhs.csv'.format(region), dtype=int, delimiter=','))
    tests.append(np.loadtxt('../../data/serology_data/{}_tests_nhs.csv'.format(region), dtype=int, delimiter=','))

In [ ]:
# Select the time points for which the death and serology data is known
deaths_times = np.arange(27, total_days+1, 1).tolist()
serology_times = np.arange(80, total_days+1, 7).tolist()

In [ ]:
# Set time-to-death using a Gamma distribution using the mean and standard deviation from the PHE paper
td_mean = 15.0
td_var = 12.1**2
theta = td_var / td_mean
k = td_mean / theta
time_to_death = gamma(k, scale=theta).pdf(np.arange(1, 31)).tolist()

# Set information
fatality_ratio = (1/100 * np.array([0.0016, 0.0016, 0.0043, 0.019, 0.08975, 0.815, 3.1, 6.05])).tolist()
time_to_death.extend([0.0] * (len(times)-30))
niu = float(gamma.rvs(1, scale=1/0.2, size=1))

sens = 0.7
spec = 0.95

## Model Fitting
### PHE with variable contacts 

In [ ]:
# Initialise inference for the model
phe_inference_var = em.inference.PheSEIRInfer(phe_model_var)

# Add model, death and tests data to the inference structure
phe_inference_var.read_model_data(susceptibles, infectives1)
phe_inference_var.read_deaths_data(deaths_data, deaths_times, time_to_death, fatality_ratio)
phe_inference_var.read_serology_data(tests, positives_data, serology_times, sens, spec)

# Run inference structure
phe_var_samples = phe_inference_var.inference_problem_setup(times, num_iter=60000)

### PHE with fixed contacts

In [ ]:
# Initialise inference for the model
phe_inference_fix = em.inference.PheSEIRInfer(phe_model_fix)

# Add model, death and tests data to the inference structure
phe_inference_fix.read_model_data(susceptibles, infectives1)
phe_inference_fix.read_deaths_data(deaths_data, deaths_times, time_to_death, fatality_ratio)
phe_inference_fix.read_serology_data(tests, positives_data, serology_times, sens, spec)

# Run inference structure
phe_fix_samples = phe_inference_fix.inference_problem_setup(times, num_iter=60000)

### Roche with NPIs

In [ ]:
# Initialise inference for the model
roche_inference_var = em.inference.RocheSEIRInfer(roche_model_var)

# Add model, death, tests and NPIs data to the inference structure
roche_inference_var.read_model_data(susceptibles, infectives)
roche_inference_var.read_deaths_data(deaths_data, deaths_times, time_to_death, fatality_ratio)
roche_inference_var.read_serology_data(tests, positives_data, serology_times, sens, spec)
roche_inference_var.read_npis_data(max_levels_npi, targeted_npi, general_npi_yes, reg_levels_npi_yes, time_changes_npi_yes, time_changes_flag_yes)

# Run inference structure
roche_var_samples = roche_inference_var.inference_problem_setup(times, num_iter=60000)

### Roche without NPIs

In [ ]:
# Initialise inference for the model
roche_inference_fix = em.inference.RocheSEIRInfer(roche_model_fix)

# Add model, death, tests and NPIs data to the inference structure
roche_inference_fix.read_model_data(susceptibles, infectives)
roche_inference_fix.read_deaths_data(deaths_data, deaths_times, time_to_death, fatality_ratio)
roche_inference_fix.read_serology_data(tests, positives_data, serology_times, sens, spec)
roche_inference_fix.read_npis_data(max_levels_npi, targeted_npi, general_npi_no, reg_levels_npi_no, time_changes_npi_no, time_changes_flag_no)

# Run inference structure
roche_fix_samples = roche_inference_fix.inference_problem_setup(times, num_iter=60000)